# Kachemak Bay Turbidity - Exploratory Data Analysis 
This notebook: <br>
1. Pulls the most recent data from ERDDAP for Seldovia and Homer Surface and Deep Water.
2. Performs Exploratory Data Analysis.

In [3]:
import requests
import pandas as pd
import os
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

### Download the most recent data (optional)

In [2]:
# URL of the webpage where the CSV link is located
data_dict = {'seldovia_swq.csv' : 'https://erddap.aoos.org/erddap/tabledap/nerrs_kacsswq.csv?time%2Csea_water_turbidity%2Csea_water_turbidity_qc_agg',
            'seldovia_dwq.csv' : 'https://erddap.aoos.org/erddap/tabledap/nerrs_kacsdwq.csv?time%2Csea_water_turbidity%2Csea_water_turbidity_qc_agg',
            'homer_dwq.csv' : 'https://erddap.aoos.org/erddap/tabledap/nerrs_kachdwq.csv?time%2Csea_water_turbidity%2Csea_water_turbidity_qc_agg',
            'homer_swq.csv' : 'https://erddap.aoos.org/erddap/tabledap/nerrs_kach3wq.csv?time%2Csea_water_turbidity%2Csea_water_turbidity_qc_agg',
            }

print()
# Send a GET request to the URL
for i, sensor in enumerate(list(data_dict)):
    response = requests.get(list(data_dict.values())[i])

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the content of the response (CSV data)
        csv_data = response.content
        
        # Specify the file name for saving the CSV
        csv_file_name = list(data_dict.keys())[i]
        
        # Write the CSV data to a file in the current directory
        with open('data/'+csv_file_name, 'wb') as csv_file:
            csv_file.write(csv_data)
        
        print(f"CSV file for sensor {i+1} saved successfully as '{csv_file_name}' in the /data directory.\n")
    else:
        print(f"Failed to download CSV file for sensor {i+1}. Status code: {response.status_code}\n")


CSV file for sensor 1 saved successfully as 'seldovia_swq.csv' in the /data directory.

CSV file for sensor 2 saved successfully as 'seldovia_dwq.csv' in the /data directory.

CSV file for sensor 3 saved successfully as 'homer_dwq.csv' in the /data directory.

CSV file for sensor 4 saved successfully as 'homer_swq.csv' in the /data directory.



### Load in CSV Files

In [4]:
# List all csv files in the data directory 
csv_files = [f for f in os.listdir('data') if f.endswith('.csv')]
print(csv_files)

['homer_dwq.csv', 'homer_swq.csv', 'seldovia_dwq.csv', 'seldovia_swq.csv']


### Data Resolution

In [3]:
def data_resolution():
    for csv in csv_files:
        # Load csv file into a DataFrame
        df = pd.read_csv(f"data/{csv}", low_memory=False)

        # # Display the first few rows to verify (UNCOMMENT FOR TESTING)
        # print(df.head())
        # print("\n")

        # Remove the first row containing units and reindex
        df = df.drop(index=0)
        df = df.reset_index(drop=True)
        # Convert the time column entries into a datetime type
        df['time'] = pd.to_datetime(df['time'])

        # Calculate the average measurement interval
        avg_interval = df['time'].diff().mean()

        # Get the first and last measurement dates
        first_measurement_date = df['time'].min()
        last_measurement_date = df['time'].max()

        # Display the results
        print(f"First Measurement Date for {csv}:", first_measurement_date.strftime('%Y-%m-%d %H:%M:%S'))
        print(f"Last Measurement Date for {csv}:", last_measurement_date.strftime('%Y-%m-%d %H:%M:%S'))
        print(f"Average Measurement Interval for {csv}: {avg_interval.components.hours} hours, {avg_interval.components.minutes} minutes")
        print("\n")


# Run the function
data_resolution()

First Measurement Date for homer_dwq.csv: 2003-01-01 09:00:00
Last Measurement Date for homer_dwq.csv: 2024-10-08 15:15:00
Average Measurement Interval for homer_dwq.csv: 0 hours, 15 minutes


First Measurement Date for homer_swq.csv: 2012-05-31 21:15:00
Last Measurement Date for homer_swq.csv: 2024-10-01 18:00:00
Average Measurement Interval for homer_swq.csv: 0 hours, 18 minutes


First Measurement Date for seldovia_dwq.csv: 2004-01-01 09:00:00
Last Measurement Date for seldovia_dwq.csv: 2024-10-02 17:30:00
Average Measurement Interval for seldovia_dwq.csv: 0 hours, 15 minutes


First Measurement Date for seldovia_swq.csv: 2004-01-01 09:00:00
Last Measurement Date for seldovia_swq.csv: 2024-10-02 17:45:00
Average Measurement Interval for seldovia_swq.csv: 0 hours, 15 minutes




### Data Quality

In [4]:
def data_quality():
    # From https://erddap.aoos.org/erddap/tabledap/nerrs_kacsswq.html
    qc_flag_meanings =  {
                        'PASS'          : 1,
                        'NOT_EVALUATED' : 2,
                        'SUSPECT'       : 3,
                        'FAIL'          : 4,
                        'MISSING'       : 9
                        }

    for csv in csv_files:
        # Load csv file into a DataFrame
        df = pd.read_csv(f"data/{csv}", low_memory=False)

        # # Display the first few rows to verify (UNCOMMENT FOR TESTING)
        # print(df.head())
        # print("\n")

        # Remove the first row containing units and reindex
        df = df.drop(index=0)
        df = df.reset_index(drop=True)

        # Make the 'sea_water_turbidity_qc_agg' column a type int instead of float
        df['sea_water_turbidity_qc_agg'] = df['sea_water_turbidity_qc_agg'].astype(int)

        # Count the occurrences of each value in the 'sea_water_turbidity_qc_agg' column
        counts = df['sea_water_turbidity_qc_agg'].value_counts()
        
        # Get the total number of rows
        total_rows = df.shape[0]

        # # Print the counts (UNCOMMENT FOR TESTING)
        # print(counts)
        print("------------------------------------------------------------")

        # counts.get(X, y) gets the value in the counts series for X and defaults to value y if X is not found
        if counts.get(4, 0) > 0: 
            pass_fail_rate = counts.get(1, 0) / counts.get(4, 0) 
        else:
            pass_fail_rate = float('inf')  # To handle the case where there are no fail entries

        print(f"Quality of {csv}:")
        print(f"Total entries: {total_rows}")
        print(f"Percentage of PASS entries: {(counts.get(1, 0) / total_rows * 100):.1f}%") 
        print(f"Percentage of NOT_EVALUATED entries: {(counts.get(2, 0) / total_rows * 100):.1f}%")
        print(f"Percentage of SUSPECT entries: {(counts.get(3, 0) / total_rows * 100):.1f}%")
        print(f"Percentage of FAIL entries: {(counts.get(4, 0) / total_rows * 100):.1f}%")
        print(f"Percentage of MISSING entries: {(counts.get(9, 0) / total_rows * 100):.1f}%")
        print(f"Pass/Fail rate: {pass_fail_rate:.1f}")


# Run the function
data_quality()

------------------------------------------------------------
Quality of homer_dwq.csv:
Total entries: 724689
Percentage of PASS entries: 74.8%
Percentage of NOT_EVALUATED entries: 16.4%
Percentage of SUSPECT entries: 4.7%
Percentage of FAIL entries: 4.1%
Percentage of MISSING entries: 0.0%
Pass/Fail rate: 18.5
------------------------------------------------------------
Quality of homer_swq.csv:
Total entries: 343585
Percentage of PASS entries: 89.4%
Percentage of NOT_EVALUATED entries: 0.0%
Percentage of SUSPECT entries: 3.3%
Percentage of FAIL entries: 7.3%
Percentage of MISSING entries: 0.0%
Pass/Fail rate: 12.2
------------------------------------------------------------
Quality of seldovia_dwq.csv:
Total entries: 712744
Percentage of PASS entries: 73.3%
Percentage of NOT_EVALUATED entries: 14.4%
Percentage of SUSPECT entries: 9.5%
Percentage of FAIL entries: 2.8%
Percentage of MISSING entries: 0.0%
Pass/Fail rate: 26.6
------------------------------------------------------------
Q

### Statistical Metrics

In [5]:
def statistical_metrics():
    for csv in csv_files:
        # Load csv file into a DataFrame
        df = pd.read_csv(f"data/{csv}", low_memory=False)

        # # Display the first few rows to verify (UNCOMMENT FOR TESTING)
        # print(df.head())
        # print("\n")

        # Remove the first row containing units and reindex
        df = df.drop(index=0)
        df = df.reset_index(drop=True)

        # Make the 'sea_water_turbidity' column a type int instead of float
        df['sea_water_turbidity'] = df['sea_water_turbidity'].astype(float)

        # Only include rows where 'sea_water_turbidity_qc_agg' is 1 (PASS)
        df_filtered = df[df['sea_water_turbidity_qc_agg'].astype(int) == 1]
    
        # Calculate statistics
        min_value = df_filtered['sea_water_turbidity'].min()
        max_value = df_filtered['sea_water_turbidity'].max()
        mean = df_filtered['sea_water_turbidity'].mean()
        median = df_filtered['sea_water_turbidity'].median()
        mode = df_filtered['sea_water_turbidity'].mode()[0]  # mode() returns a Series
        percentiles = df_filtered['sea_water_turbidity'].quantile([0.25, 0.5, 0.75])
        range_ = df_filtered['sea_water_turbidity'].max() - df_filtered['sea_water_turbidity'].min()
        variance = df_filtered['sea_water_turbidity'].var()
        std_dev = df_filtered['sea_water_turbidity'].std()
        

        # Print the results
        print(f"Quality of {csv}:")
        print(f"Lowest Value: {min_value:.2f}")
        print(f"Highest Value: {max_value:.2f}")
        print(f"Mean: {mean:.2f}")
        print(f"Median: {median:.2f}")
        print(f"Mode: {mode:.2f}")
        print(f"25th Percentile: {percentiles[0.25]:.2f}")
        print(f"50th Percentile (Median): {percentiles[0.5]:.2f}")
        print(f"75th Percentile: {percentiles[0.75]:.2f}")
        print(f"Range: {range_:.2f}")
        print(f"Variance: {variance:.2f}")
        print(f"Standard Deviation: {std_dev:.2f}")
        print("\n")


# Run the function
statistical_metrics()

Quality of homer_dwq.csv:
Lowest Value: 0.00
Highest Value: 1923.00
Mean: 3.03
Median: 2.00
Mode: 2.00
25th Percentile: 1.00
50th Percentile (Median): 2.00
75th Percentile: 3.00
Range: 1923.00
Variance: 66.50
Standard Deviation: 8.15


Quality of homer_swq.csv:
Lowest Value: 0.00
Highest Value: 286.00
Mean: 4.29
Median: 2.00
Mode: 2.00
25th Percentile: 2.00
50th Percentile (Median): 2.00
75th Percentile: 4.00
Range: 286.00
Variance: 51.11
Standard Deviation: 7.15


Quality of seldovia_dwq.csv:
Lowest Value: 0.00
Highest Value: 568.00
Mean: 1.28
Median: 1.00
Mode: 1.00
25th Percentile: 0.00
50th Percentile (Median): 1.00
75th Percentile: 1.00
Range: 568.00
Variance: 24.88
Standard Deviation: 4.99


Quality of seldovia_swq.csv:
Lowest Value: 0.00
Highest Value: 204.00
Mean: 1.19
Median: 1.00
Mode: 1.00
25th Percentile: 0.00
50th Percentile (Median): 1.00
75th Percentile: 1.00
Range: 204.00
Variance: 8.51
Standard Deviation: 2.92




### Graphs

Line Graphs of Turbidity vs Time

In [ ]:
def plot_turbidity_vs_time_individual():
    for csv in csv_files:
        # Load csv file into a DataFrame
        df = pd.read_csv(f"data/{csv}", low_memory=False)

        # # Display the first few rows to verify (UNCOMMENT FOR TESTING)
        # print(df.head())
        # print("\n")

        # Remove the first row containing units and reindex
        df = df.drop(index=0)
        df = df.reset_index(drop=True)

        # Make the 'sea_water_turbidity' column a type int instead of float
        df['sea_water_turbidity'] = df['sea_water_turbidity'].astype(float)

        # Only include rows where 'sea_water_turbidity_qc_agg' is 1 ('PASS')
        df_filtered = df[df['sea_water_turbidity_qc_agg'].astype(int) == 1]

        # Ensure 'time' is in datetime format
        df_filtered.loc[:, 'time'] = pd.to_datetime(df_filtered['time'])

        # # Plot turbidity over time
        # plt.figure(figsize=(12, 6))
        # plt.plot(df_filtered['time'], df_filtered['sea_water_turbidity'], linestyle='-', color='b')
        # plt.xlabel('Time')
        # plt.ylabel('Sea Water Turbidity (NTU)')
        # plt.title(f'Sea Water Turbidity Over Time for {csv}')
        # plt.grid(True)
        # plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
        # plt.tight_layout()  # Adjust layout to fit labels
        # plt.show()

        # Create Plotly figure
        fig = go.Figure()

        # Add the turbidity vs. time line plot
        fig.add_trace(go.Scatter(
            x=df_filtered['time'],
            y=df_filtered['sea_water_turbidity'],
            mode='lines',
            line=dict(color='blue'),
            name=f'Turbidity for {csv}'
        ))

        # Customize layout
        fig.update_layout(
            title=f'Sea Water Turbidity Over Time for {csv}',
            xaxis_title='Time',
            yaxis_title='Sea Water Turbidity (NTU)',
            xaxis=dict(tickformat='%Y-%m-%d', tickangle=45),  # Format x-axis for better readability
            template='plotly_white',
            hovermode='x unified'
        )

        # Show the plot
        fig.show()


# Run the function
plot_turbidity_vs_time_individual()

In [ ]:
def plot_turbidity_vs_time_combined():

    # Create plotly figure
    fig = go.Figure()

    for csv in csv_files:
        # Load csv file into a DataFrame
        df = pd.read_csv(f"data/{csv}", low_memory=False)

        # # Display the first few rows to verify (UNCOMMENT FOR TESTING)
        # print(df.head())
        # print("\n")

        # Remove the first row containing units and reindex
        df = df.drop(index=0)
        df = df.reset_index(drop=True)

        # Make the 'sea_water_turbidity' column a type int instead of float
        df['sea_water_turbidity'] = df['sea_water_turbidity'].astype(float)

        # Only include rows where 'sea_water_turbidity_qc_agg' is 1 ('PASS')
        df_filtered = df[df['sea_water_turbidity_qc_agg'].astype(int) == 1]

        # Ensure 'time' is in datetime format
        df_filtered.loc[:, 'time'] = pd.to_datetime(df_filtered['time'])

        # Add the turbidity vs. time line plot
        fig.add_trace(go.Scatter(
            x=df_filtered['time'],
            y=df_filtered['sea_water_turbidity'],
            mode='lines',
            line=dict(),
            name=f'Turbidity for {csv}'
        ))
    
        # Customize layout
    fig.update_layout(
        title=f'Sea Water Turbidity Over Time',
        xaxis_title='Time',
        yaxis_title='Sea Water Turbidity (NTU)',
        xaxis=dict(tickformat='%Y-%m-%d', tickangle=45),  # Format x-axis for better readability
        template='seaborn   ',
        hovermode='x unified'
    )

    # Show the plot
    fig.show()


# Run the function
plot_turbidity_vs_time_combined()

Monthly Binned Boxplots

In [6]:
def create_monthly_boxplot():
    for csv in csv_files:
        # Load csv file into a DataFrame
        df = pd.read_csv(f"data/{csv}", low_memory=False)

        # Remove the first row containing units and reindex
        df = df.drop(index=0)
        df = df.reset_index(drop=True)

        # Make the 'sea_water_turbidity' column a type int instead of float
        df['sea_water_turbidity'] = df['sea_water_turbidity'].astype(float)

        # Only include rows where 'sea_water_turbidity_qc_agg' is 1 ('PASS')
        df_filtered = df[df['sea_water_turbidity_qc_agg'].astype(int) == 1]

        # Ensure 'time' is in datetime format
        df_filtered.loc[ : , 'time'] = pd.to_datetime(df_filtered['time'], errors='raise') # This will make the column of type pandas timestamp
        assert all(isinstance(x, pd.Timestamp) for x in df_filtered['time']), "Not all values in 'time' are Timestamps"        

        # Extract the month (only) for grouping
        df_filtered.loc[ : , 'month'] = df_filtered['time'].apply(lambda x: x.strftime('%B'))
        # print(df_filtered['time'].iloc[1].strftime('%B'))

        fig = px.box(df_filtered, 
                    x='month', 
                    y='sea_water_turbidity', 
                    title='Sea Water Turbidity Distribution by Month',
                    category_orders={'month': ['January', 'February', 'March', 'April', 'May', 'June', 
                                                'July', 'August', 'September', 'October', 'November', 'December']},
                    labels={'month': 'Month', 'sea_water_turbidity': 'Sea Water Turbidity (NTU)'})

        fig.update_layout(xaxis_tickangle=-45)  # Rotate x-axis labels for better readability
        
        break
        
create_monthly_boxplot()

C:\Users\Eleonora\AppData\Local\Temp\ipykernel_22008\1905459569.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

